In [43]:
from geopy.geocoders import Nominatim
import time


conn = sqlite3.connect('data.db')


lat, lon = 37.58, -2

query = f"""
SELECT *
FROM past_data
WHERE latitude BETWEEN {lat - 1} AND {lat + 1}
AND longitude BETWEEN {lon - 1} AND {lon + 1}
"""

df = pd.read_sql_query(query, conn)
df

,latitude,longitude,time,tp,tcc,t2m,d2m,v10,u10
0,38.458,-2.934750,1992-01-01 00:00:00,0.000007,0.339057,274.178223,271.591553,-0.198302,-1.269696
1,38.458,-2.934750,1992-01-01 01:00:00,0.000005,0.375835,274.265625,271.650635,-0.188227,-1.236546
2,38.458,-2.934750,1992-01-01 02:00:00,0.000004,0.392524,273.563721,271.250488,-0.184044,-1.262031
3,38.458,-2.934750,1992-01-01 03:00:00,0.000003,0.391650,273.238281,271.174316,-0.184158,-1.242914
4,38.458,-2.934750,1992-01-01 04:00:00,0.000003,0.414954,273.257812,271.226562,-0.174994,-1.229978
...,...,...,...,...,...,...,...,...,...
374971,36.957,-1.684125,2022-12-01 19:00:00,0.000039,0.462303,289.561768,286.399414,1.689074,2.608312
374972,36.957,-1.684125,2022-12-01 20:00:00,0.000042,0.430502,289.470703,286.329590,1.385771,2.613811
374973,36.957,-1.684125,2022-12-01 21:00:00,0.000040,0.400889,289.323486,286.233887,1.151846,2.569577
374974,36.957,-1.684125,2022-12-01 22:00:00,0.000048,0.484809,289.224609,286.249268,1.047273,2.630359


In [ ]:

        
        
        
        
    ds_past = xr.open_dataset(filename, engine='cfgrib', backend_kwargs={'filter_by_keys': {'shortName': var}})
    past_ds_list.append(ds_past)

merged_ds_past = xr.merge(past_ds_list)

merged_ds_past

In [ ]:

# Create a SQLite database    
   print('Prediction_data table created successfully.')
   # Open the GRIB file with past data
   print('opening GRIB file')
   filename = "src/data/past_climate.grib"
   variables = ['tp', 'tcc', 'rh', '2t','2d','10v', '10u'] 
   datasets = {}
   # Set up Dask to use a single thread
   dask.config.set(scheduler='single-threaded')
   for var in variables:
       try:
           # Open the GRIB file with chunks
           ds = xr.open_dataset(filename, engine='cfgrib', backend_kwargs={'filter_by_keys': {'shortName': var}}, chunks={'time': 10})
           datasets[var] = ds
       except Exception as e:
           logging.error('Error opening dataset for variable %s: %s', var, e)
   #calculate relative humidity from temperature and dewpoint temperature
   def rh(dewpoint, temperature):
       return 100*(np.exp((17.625*dewpoint)/(243.04+dewpoint))/np.exp((17.625*temperature)/(243.04+temperature)))
   rh_all = rh(datasets['2d']['d2m']-273.15, datasets['2t']['t2m']-273.15)
   datasets['rh'] = xr.Dataset({'rh': xr.DataArray(rh_all, coords=datasets['2d']['d2m'].coords, dims=datasets['2d']['d2m'].dims)})
   #standardize tp to make it in the same format as the other variables 
   # Flatten the 'valid_time' coordinate from 2D to 1D
   datasets['tp']['valid_time'] = datasets['tp']['valid_time'].values.flatten()
   # Stack the 'time' and 'step' dimensions of the 'tp' variable into a single new dimension called 'total_time'
   datasets['tp']['tp'] = datasets['tp']['tp'].stack(total_time=('time', 'step'))
   # Assign the flattened 'valid_time' data to the 'total_time' coordinate
   datasets['tp'].coords["valid_time"] = ("total_time", datasets['tp']['valid_time'].data)
   # Swap the 'total_time' dimension with the 'valid_time' dimension
   datasets['tp'] = datasets['tp'].swap_dims({'total_time': 'valid_time'})
   # Drop the 'time', 'step', 'number', and 'surface' dimensions from 'tp_single_time_and_step' and assign the result to 'tp_dropped'
   tp_dropped = datasets['tp'].drop(['time','step','number','surface'])
   # Rename the 'valid_time' dimension to 'time' in 'tp_dropped' and assign the result back to 'datasets['tp']'
   datasets['tp'] = tp_dropped.rename({'valid_time': 'time'})
   # Drop all NA values in the 'time' dimension of 'datasets['tp']' and assign the result back to 'datasets['tp']'
   datasets['tp'] = datasets['tp'].dropna(dim='time', how='all')
   #TODO:check if the transpose works correctly
   # Reorder the dimensions of 'datasets['tp']' to have 'time' as the first dimension
   datasets['tp'] = datasets['tp'].transpose('time', 'latitude', 'longitude')
   print('writing to database')
   for var, ds in datasets.items():
       try:
           print('processing', var)
           with dask.config.set(**{'array.slicing.split_large_chunks': True}):
           # Convert the Dataset to a DataFrame and reset the index
               df = ds.to_dask_dataframe().reset_index()
           #print(df.dtypes)
           #print(len(df))
           # Convert the DataFrame to a Dask DataFrame
           #df = dd.from_pandas(df, npartitions=4)  
           df['variable'] = var
           print('writing', var, 'to database')
           # Write the DataFrame to the SQLite database in chunks
           # Use the variable name as the table name
           # Write the DataFrame to the SQLite database in chunks
           # Use the variable name as the table name
           print('Number of partitions:', df.npartitions)
           print('Size of DataFrame:', len(df))
           for i in range(df.npartitions):
               # Compute the partition to load the data into memory and convert it to a Pandas DataFrame
               partition_pd = df.get_partition(i).compute()
               #print(type(partition_pd))
               #print(type(partition_pd.empty)) 
                   
               # Convert timedelta columns to strings
               for col in partition_pd.columns:
                   if partition_pd[col].dtype == 'timedelta64[ns]':
                       partition_pd[col] = partition_pd[col].astype(str)
               # Check if the DataFrame is empty
               if not partition_pd.empty:
                   # Write the DataFrame to the database
                   partition_pd.to_sql(var + "_climate", conn, if_exists='append', index=False)
                   # Commit the transaction to ensure the data is saved to the database
                   conn.commit()
       except Exception as e:
           logging.error('Error writing variable %s to database: %s', var, e)
print('Database created successfully.')